# Определение стоимости автомобилей

# Введение

### Описание проекта 
Проект нацелен на оптимизацию приложения по продаже автомобилей с пробегом, в котором можно быстро узнать рыночную стоимость автомобиля по заданным параметрам.
### Цель проекта
Используя данные с параметрами и ценами автомобилей, создать модель предсказания цены машины по заданным параметрам.
### Описание данных
**Нам предоставлены исторические данные: технические характеристики, комплектации и цены автомобилей.**   


**Признаки:**
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

**Целевой признак:**
- Price — цена (евро)

**Заказчику важны:**

- качество предсказания (RMSE не более 2500);
- скорость предсказания;
- время обучения.

### План работы
1. [Загрузка и подготовка данных](#section_1)  
2. [Обучение моделей](#section_2)  
3. [Анализ результатов моделей](#section_3)
4. [Общий вывод](#section_4)

<a id='section_1'></a>
## Загрузка и подготовка данных

Произведём загрузку необходимых библиотек.

In [1]:
# анализ данных
import pandas as pd
import numpy as np
#from pandas_profiling import ProfileReport

# модели машинного обучения
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# вспомогательные средства 
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV

Загрузим файл с данными.

In [2]:
try:
    data = pd.read_csv(r'D:\projects_data\autos.csv')
except:
    data = pd.read_csv('autos.csv')

### Изучение общей информации

In [3]:
#data.profile_report(title='Отчёт по стоимости автомобилей')

Переименуем столбцы.

In [4]:
data.columns = data.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower()

In [5]:
data.head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


### Дубликаты

In [6]:
data.duplicated().sum()

4

In [7]:
data.drop_duplicates(inplace=True)

### Аномальные значения

Избавимся от аномальных значений в столбцах `price`, `registration_year`, `power`.

In [8]:
data = data[data['price'] > 100]
data = data[(data['registration_year'] > 1980) & (data['registration_year'] < 2017)]
data = data[(data['power'] > 40) & (data['power'] < 350)]

In [9]:
data['date_crawled'].astype('datetime64').max()

Timestamp('2016-04-07 14:36:58')

### Лишние столбцы

В данных присутствует столбец `number_of_pictures`, который имеет только одно значение для всех объектов. Удалим данный признак.

In [10]:
data[['date_crawled', 'date_created']].head()

,date_crawled,date_created
1,2016-03-24 10:58:45,2016-03-24 00:00:00
2,2016-03-14 12:52:21,2016-03-14 00:00:00
3,2016-03-17 16:54:04,2016-03-17 00:00:00
4,2016-03-31 17:25:20,2016-03-31 00:00:00
5,2016-04-04 17:36:23,2016-04-04 00:00:00


In [11]:
(pd.to_datetime(data['date_crawled']).dt.date == pd.to_datetime(data['date_created']).dt.date).mean()

0.9663690178968066

In [12]:
drop_list = ['number_of_pictures', 'date_crawled', 'registration_month', 'postal_code', 'date_created', 'last_seen']

In [13]:
data.drop(drop_list, axis=1, inplace=True)

### Пропуски 

Наблюдается значительное количество пропусков в столбцах `vehicle_type`, `gearbox`, `model`, `fuel_type` и `repaired`. Так как алгоритмы градиентного бустинга показывают хорошие результаты на данных с пропусками, замени отсутствуующие данные `'NaN'`.

In [14]:
data_ready = data.fillna('NaN')

### Промежуточный вывод

1. Были преобразованы данные с датами, чтобы в дальнейшем использовать их для обучения моделей.
2. Удалены дубликаты.
3. Обработаны аномальные значнеия в столбцах с данными о цене, дате регистрации и мощности автомобилей.
4. Удалены лишние признаки.
5. Обнаруженные пропуски в категориальных признаках заменены значнеием `NaN` для дальнейшего использования в алгоритмах градиентного бустинга без потери значительного количества строк.

<a id='section_2'></a>
## Обучение моделей

Создадим функцию, которая будет выдавать оценку скорости работы алгоритма и RMSE. В ней будет происходить обучение модели, выдача предсказаний и запись требуемых патаметров.

In [15]:
def model_score(model, X_train, X_valid, y_train, y_valid):
    
    start = time.time()    
    model.fit(X_train, y_train)
    training_time = time.time() - start
    
    start = time.time()  
    y_pred = model.predict(X_valid)
    predict_time = time.time() - start
     
    total_time = training_time + predict_time

    return training_time, predict_time, total_time, mean_squared_error(y_valid, y_pred, squared=False)

In [16]:
# переменная для записи результатов моделей
results = []

### Деление на выборки

Разделим данные на 3 выборки.

In [17]:
X = data_ready.drop('price', axis=1)
y = data_ready['price']

X_train, X_0, y_train, y_0 = train_test_split(X, y, test_size=0.25, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_0, y_0, test_size=0.5, random_state=12345)

In [18]:
X_train.shape, X_valid.shape, X_test.shape

((217790, 9), (36298, 9), (36299, 9))

In [19]:
cat_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']
num_features = X_train.columns.difference(cat_features).tolist()

### CatBoost

In [20]:
%%time
model = CatBoostRegressor(loss_function='RMSE', 
                          cat_features=cat_features, 
                          verbose=0,
                          random_state=12345)

params = {'learning_rate': [0.1, 0.3, 0.5],
          'iterations': [50, 100],
          'depth': [1, 5, 7, 10],
         }

cb_gs = GridSearchCV(model,
                      params,
                      cv=3,
                      scoring='neg_root_mean_squared_error',
                      verbose=0,
                      n_jobs=-1)
cb_gs.fit(X_train, y_train)
cb_gs.best_params_

CPU times: total: 59.8 s
Wall time: 3min 18s


{'depth': 10, 'iterations': 100, 'learning_rate': 0.5}

In [21]:
%%time
results.append(model_score(cb_gs.best_estimator_, X_train, X_valid, y_train, y_valid))

CPU times: total: 49.4 s
Wall time: 10.7 s


### LightGBM

Переведём категориальные признаки в тип данных `category`.

In [22]:
X_train_lgbm = X_train.copy()
X_valid_lgbm = X_valid.copy()
X_test_lgbm = X_test.copy()
for c in cat_features:
    X_train_lgbm[c] = X_train_lgbm[c].astype('category')
    X_valid_lgbm[c] = X_valid_lgbm[c].astype('category')
    X_test_lgbm[c] = X_test_lgbm[c].astype('category')

In [23]:
%%time
model = LGBMRegressor(metric='RMSE', random_state=12345)

params = {'learning_rate': [0.1, 0.3, 0.5],
          'n_estimators': [50, 100],
          'max_depth': [1, 5, 7, 10],
         }

lgbm_gs = GridSearchCV(model,
                      params,
                      cv=3,
                      scoring='neg_root_mean_squared_error',
                      verbose=0,
                      n_jobs=-1)

lgbm_gs.fit(X_train_lgbm, y_train)
lgbm_gs.best_params_

CPU times: total: 6.7 s
Wall time: 23.5 s


{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 100}

In [24]:
%%time
results.append(model_score(lgbm_gs.best_estimator_, X_train_lgbm, X_valid_lgbm, y_train, y_valid))

CPU times: total: 5.83 s
Wall time: 780 ms


### LinearRegresson

In [25]:
X_train_lr = X_train.copy()
X_valid_lr = X_valid.copy()

In [26]:
col_transformer_lr = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        cat_features
    ),
    (
        StandardScaler(), 
        num_features
    ),
    remainder='passthrough',
    verbose_feature_names_out=False
)


X_train_lr = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_lr.fit_transform(X_train_lr),
    columns=col_transformer_lr.get_feature_names_out()
)

In [27]:
X_valid_lr = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_lr.transform(X_valid_lr),
    columns=col_transformer_lr.get_feature_names_out()
)

In [28]:
lr = LinearRegression()

In [29]:
%%time
results.append(model_score(lr, X_train_lr, X_valid_lr, y_train, y_valid))

CPU times: total: 25 s
Wall time: 6.36 s


### RandomForest

In [30]:
X_train_rf = X_train.copy()
X_valid_rf = X_valid.copy()

In [31]:
col_transformer_rf= make_column_transformer(
    (
        OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), 
        cat_features
    ),
    (
        StandardScaler(), 
        num_features
    ),
    remainder='passthrough'
)

model_rf = RandomForestRegressor(random_state=12345)

pipline_rf = make_pipeline(col_transformer_rf, model_rf)

In [32]:
params = {'randomforestregressor__max_depth': [1, 5, 10, 15],
          'randomforestregressor__n_estimators': [10, 50, 100],
          'randomforestregressor__min_samples_split': [2, 5, 7],
          'randomforestregressor__min_samples_leaf': [1, 5, 7],
         }

rf_hrs = HalvingRandomSearchCV(pipline_rf,
                               params,
                               cv=3,
                               scoring='neg_root_mean_squared_error',
                               verbose=0,
                               n_jobs=-1,
                               random_state=12345)

rf_hrs.fit(X_train_rf, y_train)
rf_hrs.best_params_

C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 108 is smaller than n_iter=36298. Running 108 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'randomforestregressor__n_estimators': 50,
 'randomforestregressor__min_samples_split': 2,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__max_depth': 10}

In [33]:
%%time
results.append(model_score(rf_hrs.best_estimator_, X_train_rf, X_valid_rf, y_train, y_valid))

CPU times: total: 17 s
Wall time: 17.1 s


In [34]:
X_train_rf = X_train.copy()

col_transformer_rf= make_column_transformer(
    (
        OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), 
        cat_features
    ),
    (
        StandardScaler(), 
        num_features
    ),
    remainder='passthrough'
)

In [35]:
X_train_rf = pd.DataFrame(
    col_transformer_rf.fit_transform(X_train_rf), 
    columns= cat_features + num_features
)

In [36]:
X_train_rf.head()

,vehicle_type,gearbox,model,fuel_type,brand,repaired,kilometer,power,registration_year
0,6.0,2.0,243.0,3.0,36.0,1.0,-0.102476,-0.595044,1.349351
1,5.0,2.0,117.0,7.0,38.0,1.0,0.585498,-0.887703,-0.877269
2,6.0,2.0,121.0,7.0,30.0,2.0,-0.102476,-1.180362,0.835515
3,7.0,1.0,210.0,3.0,15.0,1.0,0.585498,-0.146300,0.321680
4,3.0,2.0,11.0,7.0,2.0,1.0,0.585498,3.619245,0.664237


<a id='section_3'></a>
## Анализ моделей

### Результаты

In [37]:
index=['CatBoost', 'LightGBM', 'LinearRegression', 'RandomForest']
columns=['training_time', 'predict_time', 'total_time', 'RMSE']
model_comparison = pd.DataFrame(results, index=index, columns=columns)
model_comparison.style.apply(lambda col: ['font-weight:bold' if x==col.min() else '' for x in col]).format('{:.2f}')

,training_time,predict_time,total_time,RMSE
CatBoost,10.61,0.12,10.73,1495.62
LightGBM,0.72,0.05,0.76,1498.11
LinearRegression,6.34,0.02,6.36,2363.61
RandomForest,16.82,0.26,17.09,1806.40


Catboost показал наилучший результат среди моделей, представленных в исследовании. LightGBM незначительно уступил в качестве (0.2%), но оказался более чем в 13 раз быстрее. Линейная регрессия попала в допустимый диапазон качества, но значительно уступила алгоритмам градиентного бустинга, а так же случайному лесу. Случайный лес оказался лучше регресси по качеству и уступил только LightGBM по времени обучения. Таким образом из-за лучшего быстродействия, к проверке на тестовой выборке проходит модель LightGBM.

### Проверка на тестовой выборке

In [38]:
y_pred = lgbm_gs.best_estimator_.predict(X_test_lgbm)
print('Финальный RMSE:', mean_squared_error(y_test, y_pred, squared=False))

Финальный RMSE: 1503.2856289983238


### Проверка на адекватность

Сделаем проверку моделей на адекватность с помощью простейшей модели предсказания.

In [39]:
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_train_lgbm, y_train)
y_pred = dummy.predict(X_test_lgbm)
print('Dummy RMSE:', mean_squared_error(y_test, y_pred, squared=False))

Dummy RMSE: 4595.698866257342


Модель прошла тест на адекватность.

<a id='section_4'></a>
## Общий вывод

В ходе исследования были обработаны и проанализированы данные о стоимости автомобилей, созданы модели предсказания стоимости, а также сделаны выводы и рекомендации.   
После проделанной работы можно выделить алгоритм LightGBM, который по совокупности результатов занимает первое место в списке моделей машинного обучения. Но так же стоит отметить результаты CatBoost, который продемонстрировал практически идентичное качество, но в 13 раз меньшую скорость выдачи предсказаний, которая очень важна для быстродействия сервиса. 

**После проделанного исследования можно сделать следующие выводы:**

1. Выявлена средняя корреляция стоимости автомобиля с мощностью и годом регистрации.
2. Выявлена средняя обратная корреляция стоимости автомобиля с пройденными километрами.
3. Алгоритм градиентного бустинга LightGBM показал лучшие результаты.
4. CatBoost продемнстрировал качество на уровне LightGBM, но с меньшим в 13 раз быстродействием.

**Рекомендации:**
1. Обратить внимание на наличие аномальных значений в столбцах, связанных с годом выпуска, мощности и цены.
2. Обратить внимание на большое количество пропусков в столбцах с категориальными переменными.
3. Обратить внимание на наличие столбца с данными о количестве фотографий. У всех автомобилей указано нулевое значение.